In [26]:
#with these we can print variables 'inline'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [27]:
#%matplotlib notebook
#%matplotlib ipympl

#with tk plots appear as pop-up windows
%matplotlib tk 

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

# 1D Random Walk

### random generator - fair draws of N steps

In [28]:
help(np.random)

Help on package numpy.random in numpy:

NAME
    numpy.random

DESCRIPTION
    Random Number Generation
    
    Use ``default_rng()`` to create a `Generator` and call its methods.
    
    =============== =========================================================
    Generator
    --------------- ---------------------------------------------------------
    Generator       Class implementing all of the random number distributions
    default_rng     Default constructor for ``Generator``
    =============== =========================================================
    
    ============================================= ===
    BitGenerator Streams that work with Generator
    --------------------------------------------- ---
    MT19937
    PCG64
    PCG64DXSM
    Philox
    SFC64
    ============================================= ===
    
    ============================================= ===
    Getting entropy to initialize a BitGenerator
    --------------------------------------------

generator used: [random.choice()](https://numpy.org/doc/1.16/reference/generated/numpy.random.choice.html#numpy.random.choice)

In [29]:
N = 10**4 #number of steps

stepsChoice = np.random.choice((-1.0,1.0), N, p=[0.5,0.5]) #I choose N time uniformly between -1 and 1

#printout
stepsChoice
len(stepsChoice)

array([-1., -1., -1., ...,  1., -1.,  1.])

10000

In [30]:
#checks on uniformity
nplus = [stepsChoice<0][0].sum() #number of forward steps
nplus
nminus = [stepsChoice>0][0].sum() #number of bacward steps
nminus

4937

5063

### trajectory construction - using cumsum

In [31]:
help(np.cumsum)

Help on function cumsum in module numpy:

cumsum(a, axis=None, dtype=None, out=None)
    Return the cumulative sum of the elements along a given axis.
    
    Parameters
    ----------
    a : array_like
        Input array.
    axis : int, optional
        Axis along which the cumulative sum is computed. The default
        (None) is to compute the cumsum over the flattened array.
    dtype : dtype, optional
        Type of the returned array and of the accumulator in which the
        elements are summed.  If `dtype` is not specified, it defaults
        to the dtype of `a`, unless `a` has an integer dtype with a
        precision less than that of the default platform integer.  In
        that case, the default platform integer is used.
    out : ndarray, optional
        Alternative output array in which to place the result. It must
        have the same shape and buffer length as the expected output
        but the type will be cast if necessary. See :ref:`ufuncs-output-type` for

In [32]:
path = stepsChoice.cumsum()
path
len(path)

array([ -1.,  -2.,  -3., ..., 126., 125., 126.])

10000

### max and min

In [33]:
maxRight = np.max(path)
i_maxRight = np.argmax(path)

maxLeft = np.min(path)
i_maxLeft = np.argmin(path)

print(f"""Extremal values of the walk:
 - {maxRight} reached at step number {i_maxRight+1}
 - {maxLeft} reached at step number {i_maxLeft+1} """)

Extremal values of the walk:
 - 162.0 reached at step number 9670
 - -44.0 reached at step number 172 


In [34]:
specialSteps = np.where(path==20)
specialSteps #steps at wich the walker was in x=20

(array([1101, 1103, 1105, 1113, 1123, 1125, 1127, 1137, 1153, 1157, 1207,
        1211, 1221, 1225, 1227, 1305, 1367, 1371, 1379, 1381, 1383, 1415,
        1425, 1427, 1429, 1431, 1439, 1443, 1447, 1449, 1453, 1455, 1457,
        1459, 1461, 1463, 1465, 1469, 1863, 1865, 2045, 2047, 2049, 2141,
        2143, 2835, 2839, 2841, 2851, 3823, 3827, 3841, 3843, 3847, 3851,
        3853, 3857, 3859, 3885, 3891, 3893, 3899, 3925, 3931, 3939, 3941,
        3949, 3955, 3957, 3963, 4885, 4887, 4889, 6535, 6547, 6549, 6555,
        6557, 6561, 6563, 6565, 6569, 6571, 6575, 6585, 6589, 6591, 6593,
        6615, 6619, 6625, 6627, 6629, 6635, 6637, 6639, 6649, 6657, 7079,
        7081, 7103, 7105, 7107, 7123, 7127, 7143, 7145, 7149, 7151, 7153,
        7169, 7173, 7181, 7185, 7187, 7191, 7207, 7209, 7211, 7841, 7843,
        7845, 7851, 7853, 7855, 7857, 7861, 7863, 7865, 7867, 7869, 7885,
        7889]),)

### plot

In [35]:
x = path                   #information about the walk
y = np.zeros(np.shape(x))  #(it is only 1D)

maxSize = max((-maxLeft,maxRight))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-maxSize, maxSize), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("x(t) [m]")
ax.set_ylabel("y(t) [m]")
plt.title("1D Random Walk")

#startup of animation
tail, *_ = ax.plot(x[0], y[0], '.', lw=0.5, markersize=9, color='red',alpha=0.4)
cursor, *_ = ax.plot([], [], 'o-', lw=2, markersize=10, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    if i>10: tail.set_data(x[i-10:i], y[i-10:i]) #we draw a vanishing tail behind the cursor
    else: tail.set_data(x[:i], y[:i])
    cursor.set_data(x[i], y[i])
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tail, cursor, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, len(x)), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 

Text(0.5, 0, 'x(t) [m]')

Text(0, 0.5, 'y(t) [m]')

Text(0.5, 1.0, '1D Random Walk')

# Statistics on 1D random walks

### array of random walks

In [51]:
nSteps = 10**3 #number of steps
nWalks = 10**3

#array in shape: walks x steps
stepsChoice = np.random.choice((-1,1), (nWalks,nSteps), p=[0.5,0.5]) #I choose N time uniformly between -1 and 1
stepsChoice

#I sum along the steps direction
path = stepsChoice.cumsum(axis=1) 
path

array([[-1, -1,  1, ...,  1,  1, -1],
       [ 1, -1,  1, ..., -1, -1,  1],
       [ 1, -1, -1, ...,  1, -1, -1],
       ...,
       [-1, -1, -1, ...,  1,  1,  1],
       [-1,  1,  1, ..., -1, -1,  1],
       [ 1,  1, -1, ...,  1,  1, -1]])

array([[ -1,  -2,  -1, ..., -22, -21, -22],
       [  1,   0,   1, ..., -32, -33, -32],
       [  1,   0,  -1, ...,  -2,  -3,  -4],
       ...,
       [ -1,  -2,  -3, ..., -16, -15, -14],
       [ -1,   0,   1, ...,  26,  25,  26],
       [  1,   2,   1, ..., -24, -23, -24]])

### furthest walker and walkers distances

In [37]:
#first i get an array of maximum distances, then get the index of the biggest
i_maxWalker = np.argmax(np.max(path,axis=1))

print(f"The walker that travelled the furthest is the {i_maxWalker}-th")

The walker that travelled the furthest is the 275-th


In [38]:
#array with max distances
walkerMaxs = np.max(path,axis=1)
walkerMaxs

array([ 30,   8,  33,  16,  39,  15,  38,   1,  52,  26,  41,  17,   9,
        23,  26,   3,  43,  18,   2,  12,  30,  42,  27,  17,  47,   8,
        30,  25,  18,  39,  40,  29,  13,  46,  -1,  53,   3,  17,  16,
        13,  15,  34,   5,   2,  58,  35,  35,  65,   3,  65,  33,  50,
        19,  11,  37,   9,  14,  14,  10,  25,   0,   1,  37,  20,  48,
        22,  17,  52,  22,  43,  35,  46,  14,  39,  70,  27,   5,  26,
        43,   2,  62,  15,  13,   4,   7,  26,  12,  32,   5,  36,  57,
        37,  36,  51,  35,  13,  -1,  22,  10,  13,  26,   8,   4,  29,
        15,   6,   2,  22,  35,  22,   6,   5,  18,   9,  -1,   2,  31,
        23,  31,  39,  36,  28,  35,  24,  11,   2,  25,   8,  17,  13,
        34,  53,   7,  30,  30,  42,   5,   8,  10,  39,  56,  44,   8,
        31,  22,  37,  20,  50,  32,   6,   4,  18,  40,  19,  22,   4,
        35,  40,  15,  12,  49,   7,  75,   2,   8,  47,   5,   6,  79,
        15,  60,  14,  41,  32,  18,  30,  20,  20,   2,  16,  6

### histograms

In [39]:
#distribution of max distance

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111)
ax.set_xlabel("Distance [m]")
ax.set_ylabel("Occurrences")
plt.title("Max distance distribution in a 1D Random Walk")

n, bins, _ = plt.hist(walkerMaxs,bins= int(len(set(walkerMaxs))*0.5),facecolor='blue',alpha=0.8)

Text(0.5, 0, 'Distance [m]')

Text(0, 0.5, 'Occurrences')

Text(0.5, 1.0, 'Max distance distribution in a 1D Random Walk')

In [40]:
allDist = path.flatten() #1D array with all distances 
allDist

finalDist = path[:,-1] #1D with final distances
finalDist

array([  1,   2,   3, ..., -26, -25, -26])

array([  12,  -16,   28,  -26,    0,    8,    8,  -14,   34,    0,   34,
        -14,  -12,   22,  -46,  -42,  -32,  -12,  -14,   -8,  -40,   24,
         26,  -10,   24,  -22,   22,  -24,    6,   38,   34,    6,  -28,
         24,  -48,   50,  -22,  -14,  -24,  -36,   -4,   34,  -42,   -8,
         46,  -24,   22,   64,  -14,   48,    8,   40,    0,    0,   26,
        -36,   -6,   -2,   -4,   14,  -20,  -78,   -6,   14,   32,    4,
          4,   44,   10,   42,   24,   44,  -16,   18,   52,  -16,  -12,
         10,   30,  -38,   26,  -26,   -4,    2,  -38,   14,    4,   16,
        -18,   28,   42,   36,   22,   46,    0,  -56,  -26,   -6,  -26,
        -12,   16,  -22,  -44,  -10,    0,  -70,  -40,    0,    8,   -4,
        -30,  -58,    6,   -4,  -20,  -64,   -2,    4,   30,   34,    4,
         14,   28,  -52,  -26,  -22,   12,  -22,  -40,  -26,   30,   28,
        -16,   10,   20,   40,  -64,  -50,  -72,   32,   50,   42,  -58,
        -10,  -16,   30,   16,   28,   10,  -38,  -

In [41]:
fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111)
ax.set_xlabel("Distance [m]")
ax.set_ylabel("Relative Occurrences")
plt.title(f"Distance distribution in N={nWalks} 1D Random Walks")


n, bins, _ = plt.hist(allDist,bins= int(len(set(allDist))),facecolor='blue', alpha=0.8, density = True, label='All distances')
n, bins, _ = plt.hist(finalDist,bins= int(len(set(finalDist))),facecolor='red', alpha=0.8, density=True, label='Final Distances')

plt.legend()
plt.show()



Text(0.5, 0, 'Distance [m]')

Text(0, 0.5, 'Relative Occurrences')

Text(0.5, 1.0, 'Distance distribution in N=1000 1D Random Walks')

### Animation of a given walk

In [42]:
n=np.random.randint(0,path.shape[0]) #specific walk I'll animate

x = path[n,:]   #information about the walk
y = np.zeros(np.shape(x))  #(it is only 1D)

maxSize = np.max(np.abs(x))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-maxSize, maxSize), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("x(t) [m]")
ax.set_ylabel("y(t) [m]")
plt.title(f"Animation of 1D Random Walk number n={n}")

#startup of animation
tail, *_ = ax.plot(x[0], y[0], '.', lw=0.5, markersize=9, color='red',alpha=0.4)
cursor, *_ = ax.plot([], [], 'o-', lw=2, markersize=10, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    if i>10: tail.set_data(x[i-10:i], y[i-10:i]) #we draw a vanishing tail behind the cursor
    else: tail.set_data(x[:i], y[:i])
    cursor.set_data(x[i], y[i])
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tail, cursor, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, len(x)), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 

Text(0.5, 0, 'x(t) [m]')

Text(0, 0.5, 'y(t) [m]')

Text(0.5, 1.0, 'Animation of 1D Random Walk number n=39')

### Static and animated plots

In [52]:
#static plot
Nmax_walk = 30  #we just animate the first 10 walks


maxSize = np.max(np.abs(path[:Nmax_walk,:]))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(0, nWalks), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("t [m]")
ax.set_ylabel("x(t) [m]")
plt.title(f"First N={Nmax_walk} 1D Random Walks")


xList = path[:Nmax_walk,:]         #coordinates of rnd walks
tList = np.ones(np.shape(xList)).cumsum(axis=1)  #y is 0 by default

plt.plot(tList,xList,linestyle='dashed',lw=0.1,marker='o',markersize=0.5)


tailList, *_ = ax.plot(tList[:,0], xList[:,0], '.', lw=0.1, markersize=8, color='red',alpha=0.2)
cursorList, *_ = ax.plot([], [], 'o', lw=1, markersize=9, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    tail=True
    if tail:
        if i>10: tailList.set_data(tList[:,i-10:i], xList[:,i-10:i]) #we draw a vanishing tail behind the cursor
        else: tailList.set_data(tList[:,:i], xList[:,:i])
    else: tailList.set_data([], [])
    cursorList.set_data(tList[:,i], xList[:,i])
        
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tailList, cursorList, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, xList.shape[1]), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 


Text(0.5, 0, 't [m]')

Text(0, 0.5, 'x(t) [m]')

Text(0.5, 1.0, 'First N=30 1D Random Walks')

In [53]:
#anim.save('animatedFigure_1.gif')

In [44]:
#animated plot
Nmax_walk = 20  #we just animate the first 10 walks


maxSize = np.max(np.abs(path[:Nmax_walk,:]))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-maxSize, maxSize), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("x(t) [m]")
ax.set_ylabel("y(t) [m]")
plt.title(f"First N={Nmax_walk} 1D Random Walks")

#startup of animation

xList = path[:Nmax_walk,:]         #coordinates of rnd walks
yList = np.zeros(np.shape(xList))  #y is 0 by default


tailList, *_ = ax.plot(xList[:,0], yList[:,0], '.', lw=0.4, markersize=8, color='red',alpha=0.2)
cursorList, *_ = ax.plot([], [], 'o', lw=1, markersize=9, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    tail=True
    if tail:
        if i>10: tailList.set_data(xList[:,i-10:i], yList[:,i-10:i]) #we draw a vanishing tail behind the cursor
        else: tailList.set_data(xList[:,:i], yList[:,:i])
    else: tailList.set_data([], [])
    cursorList.set_data(xList[:,i], yList[:,i])
        
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tailList, cursorList, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, xList.shape[1]), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 



Text(0.5, 0, 'x(t) [m]')

Text(0, 0.5, 'y(t) [m]')

Text(0.5, 1.0, 'First N=20 1D Random Walks')

In [45]:
#anim.save('1d_N20.gif') #I save the gif

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/usr/lib/python3.10/tkinter/__init__.py", line 839, in callit
    func(*args)
  File "/usr/lib/python3/dist-packages/matplotlib/backends/_backend_tk.py", line 141, in _on_timer
    super()._on_timer()
  File "/usr/lib/python3/dist-packages/matplotlib/backend_bases.py", line 1198, in _on_timer
    ret = func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1408, in _step
    still_going = super()._step(*args)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1107, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1127, in _draw_next_frame
    self._post_draw(framedata, blit)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1150, in _post_draw
    self._blit_draw

### Gaussian noise

In [46]:
nSteps = 10**3 #number of steps
nWalks = 10**3

#array in shape: walks x steps
stepsChoice = np.random.choice((-1.0,1.0), (nWalks,nSteps), p=[0.5,0.5]) #I choose N time uniformly between -1 and 1
stepsChoice

#I sum along the steps direction
path = stepsChoice.cumsum(axis=1) 
path

#generate a gaussian noise at each step
noise = np.random.normal(loc=0.0,scale=0.5,size = path.shape)
noise

path = path+noise
path

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


array([[-1.,  1., -1., ..., -1.,  1.,  1.],
       [ 1.,  1., -1., ...,  1.,  1., -1.],
       [ 1., -1., -1., ..., -1., -1.,  1.],
       ...,
       [-1.,  1.,  1., ...,  1., -1., -1.],
       [ 1., -1., -1., ..., -1.,  1., -1.],
       [-1., -1., -1., ...,  1.,  1., -1.]])

array([[ -1.,   0.,  -1., ..., -16., -15., -14.],
       [  1.,   2.,   1., ...,  52.,  53.,  52.],
       [  1.,   0.,  -1., ..., -60., -61., -60.],
       ...,
       [ -1.,   0.,   1., ...,  20.,  19.,  18.],
       [  1.,   0.,  -1., ..., -24., -23., -24.],
       [ -1.,  -2.,  -3., ...,  10.,  11.,  10.]])

array([[-0.32472423,  0.41761179, -0.03618668, ...,  0.52316021,
        -0.35240479,  0.66088218],
       [-0.09393382,  0.35959136,  0.67462954, ..., -0.70308448,
        -0.1215954 ,  0.74167003],
       [-0.55750562,  0.10481325,  0.04796417, ...,  0.81572381,
         0.75082392, -0.11790215],
       ...,
       [-0.22710388,  0.07322547, -0.19285584, ...,  0.12379851,
         0.30350881,  0.49166491],
       [-0.49992656, -0.69412773,  0.23407545, ..., -0.3246727 ,
         0.77327762,  0.46095783],
       [ 0.50794213,  0.10340297, -0.86463525, ...,  0.89718309,
         0.0014087 , -0.54372605]])

array([[ -1.32472423,   0.41761179,  -1.03618668, ..., -15.47683979,
        -15.35240479, -13.33911782],
       [  0.90606618,   2.35959136,   1.67462954, ...,  51.29691552,
         52.8784046 ,  52.74167003],
       [  0.44249438,   0.10481325,  -0.95203583, ..., -59.18427619,
        -60.24917608, -60.11790215],
       ...,
       [ -1.22710388,   0.07322547,   0.80714416, ...,  20.12379851,
         19.30350881,  18.49166491],
       [  0.50007344,  -0.69412773,  -0.76592455, ..., -24.3246727 ,
        -22.22672238, -23.53904217],
       [ -0.49205787,  -1.89659703,  -3.86463525, ...,  10.89718309,
         11.0014087 ,   9.45627395]])

In [47]:
#animated plot (just position)
Nmax_walk = 20  #we just animate the first 20 walks


maxSize = np.max(np.abs(path[:Nmax_walk,:]))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-maxSize, maxSize), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("x(t) [m]")
ax.set_ylabel("y(t) [m]")
plt.title(f"First N={Nmax_walk} 1D Random Walks")

#startup of animation

xList = path[:Nmax_walk,:]         #coordinates of rnd walks
yList = np.zeros(np.shape(xList))  #y is 0 by default


tailList, *_ = ax.plot(xList[:,0], yList[:,0], '.', lw=0.4, markersize=8, color='red',alpha=0.2)
cursorList, *_ = ax.plot([], [], 'o', lw=1, markersize=9, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    tail=True
    if tail:
        if i>10: tailList.set_data(xList[:,i-10:i], yList[:,i-10:i]) #we draw a vanishing tail behind the cursor
        else: tailList.set_data(xList[:,:i], yList[:,:i])
    else: tailList.set_data([], [])
    cursorList.set_data(xList[:,i], yList[:,i])
        
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tailList, cursorList, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, xList.shape[1]), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 

Text(0.5, 0, 'x(t) [m]')

Text(0, 0.5, 'y(t) [m]')

Text(0.5, 1.0, 'First N=20 1D Random Walks')

In [48]:
#static+animated plot position vs time
Nmax_walk = 30  #we just animate the first 10 walks


maxSize = np.max(np.abs(path[:Nmax_walk,:]))

fig = plt.figure() #new figure

#figure general setup
ax = fig.add_subplot(111, autoscale_on=False, xlim=(0, nWalks), ylim=(-maxSize, maxSize))
ax.grid()
ax.set_xlabel("t [m]")
ax.set_ylabel("x(t) [m]")
plt.title(f"First N={Nmax_walk} 1D Random Walks")


xList = path[:Nmax_walk,:]         #coordinates of rnd walks
tList = np.ones(np.shape(xList)).cumsum(axis=1)  #y is 0 by default

plt.plot(tList,xList,linestyle='dashed',lw=0.1,marker='o',markersize=0.5)


tailList, *_ = ax.plot(tList[:,0], xList[:,0], '.', lw=0.1, markersize=8, color='red',alpha=0.2)
cursorList, *_ = ax.plot([], [], 'o', lw=1, markersize=9, color='blue')
info_template = 'step number = %.d '
info_text = ax.text(0.05,0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    tail=True
    if tail:
        if i>10: tailList.set_data(tList[:,i-10:i], xList[:,i-10:i]) #we draw a vanishing tail behind the cursor
        else: tailList.set_data(tList[:,:i], xList[:,:i])
    else: tailList.set_data([], [])
    cursorList.set_data(tList[:,i], xList[:,i])
        
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tailList, cursorList, info_text

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, len(x)), interval=10, blit=True, repeat=True)

#we look at the gif
plt.show() 

Text(0.5, 0, 't [m]')

Text(0, 0.5, 'x(t) [m]')

Text(0.5, 1.0, 'First N=30 1D Random Walks')

# 3D Random Walks

In [49]:
#I define the elementary steps i take in 3D 
#(to use choice i have to cheat: i add a 1 with 0 probability that will let the function work)
elem_steps = (1,(1.0,0.0,0.0), (1.0,0.0,0.0), (1.0,0.0,0.0), (1.0,0.0,0.0), (1.0,0.0,0.0), (0.0,-1.0,0.0), (0.0,0.0,1.0), (0.0,0.0,-1.0))
probSteps=[0.0,1/8.0,1/8.0,1/8.0,1/8.0,1/8.0,1/8.0,1/8.0,1/8.0]

Nsteps=10**2

steps = np.random.choice(elem_steps,Nsteps,p=probSteps)
path = np.asarray([i for i in steps],dtype=float).cumsum(axis=0)


/tmp/ipykernel_15438/256926636.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  steps = np.random.choice(elem_steps,Nsteps,p=probSteps)


In [50]:
'''
fig = plt.figure() #new figure

maxSize = np.max(np.abs(path))

#figure general setup
ax = fig.add_subplot(111, projection='3d', autoscale_on=False, xlim=(-maxSize, maxSize), ylim=(-maxSize, maxSize), zlim=(-maxSize,maxSize))
#ax = Axes3D(fig)
ax.grid()
ax.set_xlabel("x(t) [m]")
ax.set_ylabel("y(t) [m]")
ax.set_zlabel("z(t) [m]")
plt.title("3D Random Walks")

#startup of animation

x = path[:,0]        #coordinates of rnd walks
y = path[:,1]
z = path[:,2]

#startup of animation
#tail, *_ = ax.plot(x[0], y[0],z[0], '.', lw=0.5, markersize=9, color='red',alpha=0.4)
cursor = ax.scatter3D([], [], [], 'o-', lw=2, color='blue')
#info_template = 'step number = %.d '
#info_text = ax.text(0.05, 0.95, 0.95,'', transform=ax.transAxes)

#function to update the plot
def update_plots(i):
    '''
    if i>10: 
        tail.set_data(x[i-10:i], y[i-10:i]) #we draw a vanishing tail behind the cursor
        tail.set_3d_properties(z[i-10:i])
    else: 
        tail.set_data(x[:i], y[:i])
        tail.set_3d_properties( z[:i])
    '''
    #cursor.set_data(x[i], y[i])
    #cursor.set_3d_properties((z[i]))
    cursor._offsets3d = (x[i],y[i],z[i])
    #we update te step cpunter
    #info_text.set_text(info_template % (i))
    # Return a tuple
    #return tail, cursor, info_text
    return cursor

#animation of the plot
anim = animation.FuncAnimation(fig, update_plots, np.arange(1, Nsteps), interval=10, blit=True, repeat=True)

#we look at the gif
#plt.show() 
'''

IndentationError: unexpected indent (1395067230.py, line 30)

In [ ]:
'''
#function to update the plot
def update_plots(i):
    if i>10: 
        tail.set_data(x[i-10:i], y[i-10:i]) #we draw a vanishing tail behind the cursor
        tail.set_3d_properties(z[i-10:i])
    else: 
        tail.set_data(x[:i], y[:i])
        tail.set_3d_properties( z[:i])
    cursor.set_data(x[i], y[i])
    cursor.set_3d_properties(z[i])
    #we update te step cpunter
    info_text.set_text(info_template % (i))
    # Return a tuple
    return tail, cursor, info_text

'''